### IS - Brand position
Finding Product position on Social Network (PPSN)

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import itertools
import math
import string
import json
import pymongo
from itertools import groupby

In [2]:
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=pd.core.common.SettingWithCopyWarning)

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
C:\ProgramData\Anaconda3\lib\site-packages\_pytest\mark\structures.py:378: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  @attr.s(cmp=False, hash=False)


In [3]:
import time
from datetime import datetime
today = datetime.now()

In [4]:
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [6]:
from IPython.display import clear_output

In [7]:
# Connect MongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["NIDA_PPSN_PRD"]
col_thread = mydb["NIDA_PPSN_THREAD"]
col_comment = mydb["NIDA_PPSN_COMMENT"]

In [8]:
def reduced_keyword(wtkn,redCol):
    del_list = []
    ret_wtkn = wtkn
    for each in ret_wtkn:
        if each not in redCol:
            del_list.append(each)
    ret_wtkn = [x for x in ret_wtkn if x not in del_list]
    return ret_wtkn

### 3.Process หัวกระทู้เพื่อดึง Domain (Brand & Product) ไม่ใช้โมเดลเทรนแล้วทำนาย ต้องการความถูกต้อง 100%

In [9]:
cursor_thread = col_thread.find()
df_thr_process = pd.DataFrame(cursor_thread)

In [10]:
cursor_comment = col_comment.find()
df_cmt_process = pd.DataFrame(cursor_comment)

In [11]:
# จริงๆต้องใช้เงื่อนไข t_mention_brand.str.len() != 0 และ t_mention_dairy.str.len() != 0
df_urls_milk = df_thr_process[(df_thr_process.t_mention_brand.str.len() != 0) | (df_thr_process.t_mention_dairy.str.len() != 0)]
df_urls_milk['t_mention_brand'] = df_urls_milk['t_mention_brand'].apply(lambda x: repr(set(x)))
df_urls_milk['t_mention_product'] = df_urls_milk['t_mention_product'].apply(lambda x: repr(set(x)))

In [12]:
df_join_url = pd.merge(df_urls_milk,df_cmt_process,how='inner',on='URLs')

In [13]:
df_join_url.drop(columns=['_id_x','_id_y','headline_x'],inplace=True)

In [14]:
#df_join_url = pd.merge(df_urls_milk,df_cmt_process,how='inner',on='URLs')
#df_join_url.drop(columns=['_id_x','_id_y','headline_x'],inplace=True)
df_join_url.columns = ['URLs','token_headline','t_mention_dairy','t_mention_product','t_mention_brand'
                       ,'commentId','headline','text','Retrived_date','token_text','cmt_mention_dairy'
                       ,'cmt_mention_product','cmt_mention_brand']

In [15]:
df_join_url.head(3)

,URLs,token_headline,t_mention_dairy,t_mention_product,t_mention_brand,commentId,headline,text,Retrived_date,token_text,cmt_mention_dairy,cmt_mention_product,cmt_mention_brand
0,https://pantip.com/topic/39868603,"[นม, เมจิ, รสหวาน, หา]",[นม],{'รสหวาน'},{'เมจิ'},1,นมเมจิรสหวานทำไมหายากจังคะ,เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,2020-11-29 00:13:26.571,"[ปี, 7-Eleven, ยังมี, นม, เมจิ, รสหวาน, ขาย, อ...","[นม, นม]","[รสหวาน, รสหวาน]",[เมจิ]
1,https://pantip.com/topic/39868603,"[นม, เมจิ, รสหวาน, หา]",[นม],{'รสหวาน'},{'เมจิ'},2,นมเมจิรสหวานทำไมหายากจังคะ,เบาหวานครับ แฮ่ๆ,2020-11-29 00:13:26.571,[เบาหวาน],[],[],[]
2,https://pantip.com/topic/39868603,"[นม, เมจิ, รสหวาน, หา]",[นม],{'รสหวาน'},{'เมจิ'},3,นมเมจิรสหวานทำไมหายากจังคะ,หาซื้อไม่ได้นี่คิดได้ 2 อย่างนะ. คนไม่ค่อย...,2020-11-29 00:13:26.571,"[หา, ซื้อไม่ได้, คิดได้, , คน, กินกัน, ไม่, ทำ...",[],[รสหวาน],[]


EDA#2

In [16]:
df_view = df_urls_milk[['URLs','t_mention_product']]
df_view.column = ['URLs','t_mention_product']
df_view_count = df_view.groupby('t_mention_product').count()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [17]:
df_c_urls_milk = df_cmt_process[(df_cmt_process.cmt_mention_brand.str.len() != 0) | (df_cmt_process.cmt_mention_product.str.len() != 0)]
df_c_urls_milk['cmt_mention_brand'] = df_c_urls_milk['cmt_mention_brand'].apply(lambda x: repr(set(x)))
df_c_urls_milk['cmt_mention_product'] = df_c_urls_milk['cmt_mention_product'].apply(lambda x: repr(set(x)))

In [18]:
df_view2 = df_c_urls_milk[['URLs','cmt_mention_product']]
df_view2.column = ['URLs','cmt_mention_product']
df_view_count2 = df_view2.groupby('cmt_mention_product').count()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [20]:
print('จำนวนกระทู้',df_thr_process.shape[0])
print('จำนวน comment',df_cmt_process.shape[0])
print('จำนวนกระทู้พูดถึง brand หรือ มีนมเกี่ยวข้อง อย่างใดอย่างหนึ่ง',df_urls_milk.shape[0])
print('จำนวนกระทู้มีระบุตัว product',df_view[(df_view.t_mention_product!="{'นม'}")&(df_view.t_mention_product!="set()")].shape[0])
print('จำนวน comment พูดถึง brand หรือมีนมเกี่ยวข้องง อย่างใดอย่างหนึ่ง',df_c_urls_milk.shape[0])
print('จำนวน comment มีระบุตัว product',df_view2[(df_view2.cmt_mention_product!="{'นม'}")&(df_view2.cmt_mention_product!="set()")].shape[0])
print('odd product/นม กระทู้',round(df_view[(df_view.t_mention_product!="{'นม'}")&(df_view.t_mention_product!="set()")].shape[0]/df_urls_milk.shape[0],2))
print('odd product/นม comment',round(df_view2[(df_view2.cmt_mention_product!="{'นม'}")&(df_view2.cmt_mention_product!="set()")].shape[0]/df_c_urls_milk.shape[0],2))
print('หมายเหตุ : ไม่ได้ impute ขั้วกระทู้เข้า comment')

จำนวนกระทู้ 1515
จำนวน comment 62710
จำนวนกระทู้พูดถึง brand หรือ มีนมเกี่ยวข้อง อย่างใดอย่างหนึ่ง 467
จำนวนกระทู้มีระบุตัว product 52
จำนวน comment พูดถึง brand หรือมีนมเกี่ยวข้องง อย่างใดอย่างหนึ่ง 5220
จำนวน comment มีระบุตัว product 2449
odd product/นม กระทู้ 0.11
odd product/นม comment 0.47
หมายเหตุ : ไม่ได้ impute ขั้วกระทู้เข้า comment


จาก Issue นี้จะพบว่าทำ Product position ได้ยาก แต่ทำ Brand position แทนได้ เนื่องจากสัดส่วนของข้อความที่มีการพูดถึง brand+flavor 
มีน้อยกว่าครึ่ง จึงให้ใช้วิธีเอา brand ของหัวกระทู้แปะไปในทุก comment แทน เพื่อทำ Brand Position ส่วน flavor จะกลายเป็น attribute ไป

In [21]:
df_view_count2.sort_values(by='URLs',ascending=False)

,URLs
cmt_mention_product,
set(),2771
{'ช็อกโกแลต'},356
{'รสจืด'},320
{'ไขมันต่ำ'},203
{'รสหวาน'},154
...,...
"{'รสธรรมชาติ', 'ไขมัน 0%', 'ไขมันต่ำ'}",1
"{'รสธรรมชาติ', 'ไขมัน 0%', 'บัลแกเรีย', 'ไขมันต่ำ'}",1
"{'รสจืด', 'สตรอว์เบอร์รี', 'ไขมันต่ำ'}",1


###  4. (ยกเลิกแต่เก็บ code) คัดเลือกคำมาเป็น Attribute ด้วย TF-IDF

In [ ]:
#import gensim
#from gensim.models import TfidfModel

In [ ]:
# Create Bag of Word & calculate TF-IDF
#dictionary = gensim.corpora.Dictionary(df_join_url['token_text'])
#gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in df_join_url['token_text']]
#model = TfidfModel(gensim_corpus)
#vector = model[gensim_corpus]
#word_frequencies = [[(dictionary[id], tfidf_val) for id, tfidf_val in couple] for couple in vector]

In [ ]:
#for i in range(0,len(word_frequencies)):
#    word_frequencies[i].sort(key=lambda tup:(-tup[1], tup[0]))  # SORT LIST of TUPLE DESCENDINGLY
#    word_frequencies[i] = word_frequencies[i][:10]   

In [ ]:
# List complehension : get the first element of each tuple in a list
# https://www.kite.com/python/answers/how-to-get-the-first-element-of-each-tuple-in-a-list-in-python
#first_tuple_elements = [[a for a,tf in couple] for couple in word_frequencies]

In [ ]:
#first_tuple_elements

### 4. คัดเลือกคำมาเป็น Attribute ด้วย LDAvis

https://radimrehurek.com/gensim/models/ldamodel.html

In [25]:
# Create Bag of Word & calculate TF-IDF
dictionary = gensim.corpora.Dictionary(df_join_url['token_text'])
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in df_join_url['token_text']]

In [26]:
num_topics = 8
chunksize = 5000                  # size of the doc looked at every pass
iterations = 50
eval_every = 1                    # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize \
                                     ,alpha='auto', eta='auto',iterations=iterations \
                                     ,num_topics=num_topics \
                                     ,eval_every=eval_every)

Wall time: 15.9 s


In [27]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary, R=40, lambda_step=0.05)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.039081  0.059029       1        1  38.430325
0      0.027606  0.040527       2        1  11.916821
4     -0.002741  0.030323       3        1  11.595761
1     -0.008542  0.002421       4        1  10.952528
6      0.056974 -0.002914       5        1   8.781860
5      0.066017 -0.027352       6        1   8.533211
7     -0.082700 -0.022632       7        1   6.000239
2     -0.017532 -0.079402       8        1   3.789256, topic_info=      Term         Freq        Total Category  logprob  loglift
24     กิน  4184.000000  4184.000000  Default  40.0000  40.0000
51     ชอบ  1900.000000  1900.000000  Default  39.0000  39.0000
40    ดื่ม  1153.000000  1153.000000  Default  38.0000  38.0000
7       นม  6117.000000  6117.000000  Default  37.0000  37.0000
31      ทำ  1656.000000  1656.000000  Default  36.0000  36.0000
..     ...          ...          ...      ...      ...      ...
180    รู้    28.829910   901.549938   Topic8  -5.6480  -0.1697
646  นมวัว    26.541437   634.277566   Topic8  -5.7307   0.0992
3      ขวด    26.483377   783.031268   Topic8  -5.7329  -0.1137
242    ทาน    26.917594  1008.091130   Topic8  -5.7166  -0.3500
546    แม่    25.755083  1579.219967   Topic8  -5.7608  -0.8431

[853 rows x 6 columns], token_table=      Topic      Freq                 Term
term                                      
23        1  0.678158                     
23        2  0.024932                     
23        3  0.123237                     
23        4  0.051645                     
23        5  0.047549                     
...     ...       ...                  ...
88        7  0.061993                  ไหม
88        8  0.091514                  ไหม
8778      6  0.984411              ไอโอดีน
9324      4  0.832732              ๆๆๆๆๆๆๆ
9621      8  0.750040  ๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆ

[2156 rows x 3 columns], R=40, lambda_step=0.05, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 5, 2, 7, 6, 8, 3])

### แสดงคำในโมเดลในแต่ละ Lambda

In [ ]:
#Get the most relevant topics to the given word.
# Set probability = 0.0001

#def get_terms_topics(term_text, d, model, minProb=0.0001):
#    listofTup = []
#    for tt_couple in model.get_term_topics(dictionary.token2id[term_text], minimum_probability=minProb):
#        id_, prob = tt_couple
#        listofTup.append((d[id_], prob))
#    df = pd.DataFrame(listofTup)
#    df.insert(0,'Term',value=term_text)
#    df.columns = ['Term','topic','prob']
#    return df.sort_values(by='prob', ascending=False)

In [ ]:
# องค์ประกอบพื้นฐาน ตาม master ที่ใช้ตัดคำ และ Generalized
#get_terms_topics('ซื้อ',dictionary,model)
#get_terms_topics('ขาย',dictionary,model)

In [54]:
#Get the most relevant topics to the given word.
# Set probability = 0.0001

def get_topics_terms(topic, d, model):
    listofTup = []
    for tt_couple in model.get_topic_terms(topic, topn=10000):
        id_, prob = tt_couple
        listofTup.append((d[id_], prob))
    df = pd.DataFrame(listofTup)
    df.insert(0,'Topic',value=d[topic])
    df.columns = ['Topic','term','prob']
    return df.sort_values(by='prob', ascending=False)

In [55]:
# ไล่เช็คทีละ Topic โดยเลือกจาก Top100 Prob แต่ละกลุ่ม และดูตัวที่สนใจ (ตามทบทวนวรรณกรรม)
# สุดที่ num_topics = 8 กลุ่ม

for i in range(0,8):
    if i ==0:
        df = get_topics_terms(i, dictionary, model)
    else:
        df = df.append(get_topics_terms(i, dictionary, model))

In [58]:
df.drop_duplicates(subset='term', keep='first', inplace=True, ignore_index=True)

In [59]:
df

,Topic,term,prob
0,7-Eleven,นม,0.048794
1,7-Eleven,ไม่,0.031047
2,7-Eleven,ได้,0.021429
3,7-Eleven,กิน,0.012577
4,7-Eleven,นมสด,0.010745
...,...,...,...
14312,นม,ลำน้ำ,0.000011
14313,นม,กาล,0.000011
14314,นม,รดน้ำ,0.000011
14315,นม,กึ่กกก,0.000011


In [61]:
df[['term','prob']].to_excel('uniqueText_Lev3.xlsx')

สรุปคำที่จะนำมาเป็น Attribute (รวม link ที่เป็นเคสยกเว้น)

In [9]:
execption_urls = ['https://pantip.com/topic/30833944','https://pantip.com/topic/30105850','https://pantip.com/topic/35439062']

In [10]:
attrributes = ['hiq','lactose','เกินไป','เข้มข้น','เจ้าหน้าที่','เจือจาง','เด็ก','เต้าฮวย','เติบโต','เติม','เท','เนย','เนื้อ'
               ,'เปรี้ยว','เปลี่ยนไป','เพื่อน','เมลาโทนิน','เย็น','เยี่ยม','เลี้ยง','เลี้ยงดู','เลี้ยงลูก','เวย์','เสี่ยง','แกลลอน','แก้ว','แข็งแรง','แคลเซียม','แคลเซี่ยม','แช่เย็น'
               ,'แดรี่โฮม','แถม','แนะนำ','แพ้','แพ็ค','แพง','แพ้นม','แพ้นมวัว','แพะ','แฟน','แม่','แมว','โกโก้','โค','โฆษณา','โด๊ป','โปรโมชั่น','โปรตีน','โรงเรียน'
               ,'โรงพยาบาล','โอเมก้า','ในประเทศ','ให้นม','ไขมัน','ไขมันต่ำ','ไม่ชอบ','ไม่อร่อย','ไอโอดีน','กระดูก','กรีก','กล้วย','กล่อง','กลัว','กลิ่น','กาแฟ'
               ,'การผลิต','กิจกรรม','ข้น','ขวด','ครอบครัว','ครีม','คลอด','คลอดลูก','ความเจ็บปวด','ความมัน','ความร้อน','ความสุข','ค่าขนส่ง','คาว','คุณแม่'
               ,'คุณค่า','คุณภาพ','คุ้ม','งาน','จิตรลดา','จืด','จุลินทรีย์','ชง','ชอบ','ชาเขียว','ชาไข่มุก','ชานมไข่มุก','ชิม','ซื้อ','ญี่ปุ่น','ดั้งเดิม','ต้ม','ตอนเช้า'
               ,'ตอนกลางวัน','ตั้งครรภ์','ตั้งท้อง','ติดใจ','ตู้เย็น','ถ้วย','ท้อง','ท้องเสีย','ท้องผูก','นมเปรี้ยว','นมโคแท้','นมกล่อง','นมข้น','นมควาย','นมถั่วเหลือง'
               ,'นมผง','นมวัว','นมสด','นอน','น้ำตาล','น้ำนม','น้ำนมโค','น้ำผลไม้','น้ำผึ้ง','บรรจุ','บริจาค','บูด','ปริมาณ','ปรุงแต่ง','ปลอดภัย','ปวดท้อง','ผง'
               ,'ผลไม้','ผลิต','ผสม','ผัก','ผู้หญิง','ฝา','พกพา','พนักงาน','พยาบาล','พลังงาน','พ่อ','พาสเจอร์ไรส์','ฟาร์ม','มะนาว','มะพร้าว','มะลิ','มีประโยชน์'
               ,'มีปัญหา','มีลูก','มื้อ','ยาคูลท์','รสกลมกล่อม','รสกาแฟ','รสจืด','รสชาติ','รสธรรมชาติ','รสหวาน','ราคาแพง','ร่างกาย','ร้าน','ลดราคา','ลอง','ลำไส้'
               ,'ลูก','ลูกค้า','ลูกชาย','ลูกสาว','วัว','วิตามิน','ส่วนผสม','สะดวก','สารอาหาร','สิ่งแปลกปลอม','สีน้ำเงิน','สีส้ม','สุดยอด','สูตร','หมดอายุ','หมอ','หมา'
               ,'หวาน','หอม','ห้าง','หางนม','หาซื้อ','อยากสูง','อร่อย','อ้วก','ออกกำลังกาย','อาหาร','อึ','อุณหภูมิ','อุ่น','ไขมันทรานส์','ดาราเดลี่','แดรี่ฟาร์ม','แดรี่ควีน'
               ,'ยืดเส้นยืดสาย','ฟุตบอล','กระโดดเชือก','มวลกระดูก','ของแถม','ราคาสูง','บรรจุภัณฑ์','ความแข็งแรง','แข็งแรง','พรีเซ็นเตอร์','แลคโตส','ขายไม่ดี'
               ,'แพคคู่','ค่าจัดส่ง','shelf life','พนักงานขายนม','ซื้อประจำ','หายาก','คุมอาหาร','นักวิ่ง','ร้านนมสด','ดูแลสุขภาพ','เจมส์ จิ','เจมส์จิ','ณเดช','ณเดชน์'
               ,'อยากสูง','ส่วนสูง','สูงขึ้น','รักษามะเร็ง','รักษาเบาหวาน','ทำฟอง','ตีฟอง','โฟมนม','มื้อเช้า']

stores = ['tops','makro','lotus','bigc','7-Eleven']

products = ['สตรอว์เบอร์รี','ช็อกโกแลต','รสกาแฟ','รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','ไฮโปรตีน','อัลมอนด์'
                ,'รสกล้วย','grass fed','นมฟรีแลคโตส','เมจิโกลด์','นมฮอกไกโด','เบดไทม์','ดาร์คช็อกโกแลต','ไฮแคลเซียม'
                ,'คาราเมล','มอลต์','เมล่อน','ชาเขียวมัจฉะ','บัลแกเรีย','รสธรรมชาติ','รสกลมกล่อม','ซากุระ','วิปครีม']

milk_kind=['นม','นมข้น','นมจืด','นมสด','กินนม','ดื่มนม','ขวดนม','นมวัว','นมกล่อง','ผลิตภันท์นม','น้ำนมโค'
                     ,'โยเกิร์ต','นมเปรี้ยว','uht','นมถั่วเหลือง','นมผง','พาสเจอร์ไรส์','nondairy','non dairy']

#  EDA ใน Excel ได้ flavor 68 ตัว (ถ้าดึงยี่ห้ออื่นก็เพิ่มอีก)
avai_flavs = ['เมจิเมจิโกลด์','เมจิเมล่อน','เมจิไขมัน 0%','เมจิไขมันต่ำ','เมจิไฮโปรตีน','เมจิช็อกโกแลต','เมจิชาเขียวมัจฉะ','เมจิดาร์คช็อกโกแลต'
 ,'เมจินมฟรีแลคโตส','เมจิบัลแกเรีย','เมจิมอลต์','เมจิรสกลมกล่อม','เมจิรสกล้วย','เมจิรกาแฟ','เมจิรสจืด','เมจิรสธรรมชาติ','เมจิรสหวาน'
 ,'เมจิสตรอว์เบอร์รี','เมจิอัลมอนด์','เอ็มมิลค์นมฟรีแลคโตส','เอ็มมิลค์รสจืด','แดรี่โฮมgrass fed','แดรี่โฮมเบดไทม์','แดรี่โฮมช็อกโกแลต'
 ,'แดรี่โฮมรสกล้วย','แดโฮมรสจืด','แดรี่โฮมรสหวาน','แดรี่โฮมสตรอว์เบอร์รี','แมคโนเลียไขมันต่ำ','แมคโนเลียช็อกโกแลต','แมคโนเลียรสจืด'
 ,'โชคชัยไขมันต่ำ','โชคชัยช็อกโกแลต','โชคชัยรสกาแฟ','โชคชัยรสจืด','โชคชัยสตรอว์เบอร์รี','ฟร์โมสต์ไขมัน 0%','โฟร์โมสต์ไขมันต่ำ'
 ,'โฟร์โมสต์คาราเมล','โฟร์โมสต์ช็อกโกแลต','โฟร์โมสต์รสกาแฟ','โฟร์โมสต์รสจืด','โฟร์โมสต์สตรอว์เบอร์รี','ไทยเดนมาร์คช็อกโกแลต'
 ,'ไทยเดนมาร์ครสกาแฟ','ไทยเดนมาร์ครสจืด','ไทยเดนมาร์ครสหวาน','ไทยเดนมาร์คสตรอว์เบอร์รี','คาเนชั่นรสจืด','จิตรลดาช็อกโกแลต'
 ,'จิตรลดารสจืด','จิตรลดารสหวาน','จิตรลดาสตรอว์เบอร์รี','ดัชมิลล์ไขมัน 0%','ดัชมิลล์ไขมันต่ำ','ดัชมิลล์ไฮโปรตีน','ดัชลล์อกกแลต'
 ,'ดัชมิลล์มอลต์','ดัชมิลล์รสกาแฟ','ดัชมิลล์รสจืด','ดัชมิลล์สตรอว์เบอร์รี','หนองโพไขมัน 0%','หนองโพไขมันต่ำ','หนองโพช็อกโกแลต'
 ,'หนองโพรสกาแฟ','หนองโพรสจืด','หนองโพรสหวาน','หนองโพสตรอว์เบอร์รี']

reduceCol = attrributes + stores + products
reduceCol_all = attrributes + stores + milk_kind + products

In [11]:
print(len(attrributes))
print(len(reduceCol))

232
263


### 5.สร้าง (Reduce) Bag of word ด้วย dictionary.doc2bow จัดลง dataframe

In [12]:
cursor_thread = col_thread.find()
df_thr_process = pd.DataFrame(cursor_thread)
cursor_comment = col_comment.find()
df_cmt_process = pd.DataFrame(cursor_comment)

In [13]:
# คัด headline ที่พูดถึง domain นม
df_urls_milk = df_thr_process[(df_thr_process.t_mention_dairy.str.len() != 0) | (df_thr_process.URLs.isin(execption_urls))]
#df_urls_milk = df_thr_process[(df_thr_process.URLs.isin(execption_urls))]

In [14]:
# เมื่อ join กันแล้ว คอมเม้นทุกคอมเม้นจะอยู่ใน domain นมทั้งหมด
df_join_url = pd.merge(df_urls_milk,df_cmt_process,how='inner',on='URLs')

In [15]:
df_join_url.drop(columns=['_id_x','_id_y','headline_y'],inplace=True)
df_join_url.rename(columns={"headline_x":"headline"},inplace=True)

In [16]:
df_join_url['token_text_reduce'] = df_join_url['token_text'].apply(lambda x: reduced_keyword(x, reduceCol))

### 6.ทำ Topic Modeling จาก Reduce BoW

6.1 สร้าง Bag of word

In [28]:
#dictionary = gensim.corpora.Dictionary(df_join_url['token_text_reduce'])
#gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in df_join_url['token_text_reduce']]
#word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]
#id2word = dictionary.id2token

6.2 หา Optimal Dimension

In [18]:
#mlp = r'E:/master_BADS/IS_BADS/Product-Position-SNA/mallet-2.0.8/'
#mlp = r'D:/development_TestCase/Product-Position-SNA/mallet-2.0.8/'
#import os
#from gensim.models.wrappers import LdaMallet
#from gensim.models import CoherenceModel
#os.environ.update({'MALLET_HOME':mlp})

In [19]:
#mallet_path = "D:/development_TestCase/Product-Position-SNA/mallet-2.0.8/bin/mallet"
#mallet_path = "E:/master_BADS/IS_BADS/Product-Position-SNA/mallet-2.0.8/bin/mallet"
#limit=20; start=1; step=1;
#coherence_values = []
#model_list = []
#for num_topics in range(start, limit, step):
#    model = LdaMallet(mallet_path, corpus=gensim_corpus, num_topics=num_topics, id2word=id2word)
#    model_list.append(model)
#    coherencemodel = CoherenceModel(model=model, texts=df_join_url['token_text_reduce'], dictionary=dictionary, coherence='c_v')
#    coherence_values.append(coherencemodel.get_coherence())

In [20]:
#coherence_values

In [21]:
#model.show_topics(formatted=False)

In [29]:
# Show graph
#x = range(start, limit, step)
#plt.plot(x, coherence_values)
#plt.xlabel("Num Topics")
#plt.ylabel("Coherence score")
#plt.legend(("coherence_values"), loc='best')
#plt.show()

In [30]:
# Print the coherence scores
#for m, cv in zip(x, coherence_values):
#    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

6.3 สร้าง LdaModel ตามกลุ่มที่หาได้

In [31]:
#num_topics = 9
#chunksize = 6000                  # size of the doc looked at every pass
#iterations = 50
#eval_every = 1                    # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
#id2word = dictionary.id2token

#%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize \
#                                     ,alpha='auto', eta='auto',iterations=iterations \
#                                     ,num_topics=num_topics \
#                                     , eval_every=eval_every)

In [32]:
#pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary, R=40, lambda_step=0.1)
#pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary, R=40, lambda_step=0.1, mds='tsne')

In [56]:
topics = [[(term, round(wt, 3)) for term, wt in model.show_topic(n, topn=20)] for n in range(0, model.num_topics)]
# set column width
pd.set_option('display.max_colwidth', -1)
topics_df = pd.DataFrame([', '.join([term for term, wt in topic]) for topic in topics], columns = ['Terms per Topic'], index=['Topic'+str(t) for t in range(1, model.num_topics+1)] )
topics_df

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


,Terms per Topic
Topic1,"เด็ก, อร่อย, ชอบ, ผสม, จืด, พ่อ, แม่, วัว, หอม, รสหวาน, โรงเรียน, ซื้อ, ผลิต, นมกล่อง, นมวัว, พาสเจอร์ไรส์, ช็อกโกแลต, รสชาติ, กลิ่น, แมว"
Topic2,"กล่อง, นมสด, ชอบ, ไขมัน, น้ำตาล, โค, นมผง, ผสม, ร้าน, ซื้อ, ขวด, อาหาร, ไขมันต่ำ, จืด, ช็อกโกแลต, ผง, เด็ก, พนักงาน, พาสเจอร์ไรส์, หวาน"
Topic3,"ลูก, แม่, ลอง, เด็ก, นมผง, โปรตีน, ชอบ, นอน, มีลูก, หมอ, คลอด, คุณแม่, กลัว, นมวัว, เลี้ยงลูก, ซื้อ, เลี้ยง, สารอาหาร, ให้นม, กลิ่น"
Topic4,"ท้อง, ลูก, อาหาร, โปรตีน, ท้องเสีย, แพ้, ชอบ, แคลเซียม, นมวัว, นมถั่วเหลือง, นมเปรี้ยว, รสชาติ, ร่างกาย, ปริมาณ, เนื้อ, น้ำตาล, สารอาหาร, แม่, ไขมัน, จืด"
Topic5,"ชอบ, กาแฟ, อร่อย, ลอง, หอม, สูตร, เข้มข้น, ซื้อ, เด็ก, ลูกค้า, หวาน, กล่อง, สตรอว์เบอร์รี, เลี้ยงดู, แนะนำ, รสกาแฟ, 7-Eleven, ญี่ปุ่น, โปรโมชั่น, รสธรรมชาติ"
Topic6,"แม่, งาน, ฝา, ฟาร์ม, กลิ่น, เพื่อน, รสหวาน, เด็ก, โฆษณา, นมผง, มะลิ, ขวด, ชอบ, ครอบครัว, เลี้ยง, ชง, หอม, นมวัว, วัว, แนะนำ"
Topic7,"ซื้อ, ขวด, จิตรลดา, กล่อง, 7-Eleven, แก้ว, ลอง, ชอบ, ห้าง, เท, อร่อย, หมอ, นอน, นมผง, แพ็ค, นมกล่อง, แพง, ลูก, จุลินทรีย์, พาสเจอร์ไรส์"
Topic8,"อร่อย, ชอบ, รสจืด, นมวัว, ลอง, ซื้อ, กล่อง, แพง, ช็อกโกแลต, อาหาร, รสชาติ, นมกล่อง, กลัว, หวาน, มีปัญหา, คุณค่า, นมถั่วเหลือง, แนะนำ, ไขมันต่ำ, สีน้ำเงิน"
Topic9,"นมผง, นมสด, กล่อง, หวาน, จืด, น้ำตาล, เติม, ชอบ, รสชาติ, น้ำนม, ลอง, ผสม, โค, ครีม, สูตร, ร้าน, สารอาหาร, ร่างกาย, กาแฟ, นมข้น"
